# ДЗ 2. Машинное обучение
## Камаев Виктор БИБ214

In [222]:
import numpy as np
import pandas as pd

## Задание 1. Байесовская классификация

#### 0. Откроем файл names.csv

In [223]:
df = pd.read_csv('baby-names.csv', encoding='utf-8')
df.head()

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy


Прежде чем работать с данными, целевую переменную нужно закодировать. Так как это пол - бинарный признак, то можно воспользоваться обычным `LabelEncoder`

In [224]:
from sklearn.preprocessing import LabelEncoder

sex_encoder = LabelEncoder()
df['sex'] = sex_encoder.fit_transform(df['sex'])

df.head()

,year,name,percent,sex
0,1880,John,0.081541,0
1,1880,William,0.080511,0
2,1880,James,0.050057,0
3,1880,Charles,0.045167,0
4,1880,George,0.043292,0


#### 1. Разделим данные

In [225]:
from sklearn.model_selection import train_test_split

X, y = df['name'], df['sex']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 2. Обучение

Немного (почти) переписал функции из семинара. Имхо так с ними работать и понимать их намного удобнее

In [226]:
from collections import defaultdict
from math import log
from typing import Callable

class NaiveBayesClassifier:
    def __init__(self, feature_extraction_func: Callable, classification_func: Callable) -> None:
        self.feature_extraction_func = feature_extraction_func
        self.classification_func = classification_func


    def fit(self, X_train: list[str], y_train: list[str]):

        classes, freqs = defaultdict(int), defaultdict(int)

        for sample, label in zip(X_train, y_train):
            classes[label] += 1  # count classes frequencies

            for feat in self.feature_extraction_func(sample):
                freqs[label, feat] += 1          # count features frequencies

        for label, feat in freqs:                # normalize features frequencies
            freqs[label, feat] /= classes[label]

        for c in classes:                       # normalize classes frequencies
            classes[c] /= len(X_train)

        self.classes = classes
        self.freqs = freqs


    def predict(self, X_test: list[str]):

        res = np.zeros(len(X_test))

        for i, sample in enumerate(X_test):
            feats = self.feature_extraction_func(sample)

            res[i] = self.classification_func(self.classes, self.freqs, feats)

        return res
        

Такая реализация позволяет легко менять функции для получения признаков из имени, а также функции выбора класса

В качестве первой функции выбора возьму функцию из семинара. Но обычный копипаст ломается на том, что может оказаться так, что набор (<последняя буква>, <класс>) есть либо только в тренировочной выборке, либо только в тестовой

Чтобы бороться с этим буду смотреть на частоту, получаемую из словаря `freqs`. Если она нулевая (defauldict возвращает 0 по стандарту), то будем выдавать минимальный класс (по числу объектов тестовой выборки)

In [227]:
def clf_func_1(classes, freqs, features):
    min_res, min_class = 1e9, None
    
    for label in classes:

        res = -log(classes[label])

        for feat in features:
            if freqs[label, feat] == 0:  # handle lack of data
                return min(classes, key=lambda l: classes[l])
            
            res += -log(freqs[label, feat])

        if res < min_res:
            min_class = label
            min_res = res
            
    return min_class


def extr_func_last_letter(sample: str) -> list[str]:
    return (sample[-1])  # just last letter

Обучим

In [228]:
clf1 = NaiveBayesClassifier(extr_func_last_letter, clf_func_1)

clf1.fit(X_train, y_train)

#### 3. Предсказания

Средняя доля правильных ответов - число совпадений / число объектов

In [229]:
pred = clf1.predict(X_test)

score = sum(x == y for x, y in zip(y_test, pred)) / len(pred)
score

0.7766795865633075

Видим, что средняя доля правильных ответов 0.78. Это довольно неплохо

Обычно в задачах классификации берут метрики точности полноты (precision & recall), а также их композиции F1-score и ROC AUC

Для простоты буду выводить `classification_report`

In [230]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred, 
                            target_names=sex_encoder.classes_))

              precision    recall  f1-score   support

         boy       0.75      0.84      0.79     38668
        girl       0.81      0.72      0.76     38732

    accuracy                           0.78     77400
   macro avg       0.78      0.78      0.78     77400
weighted avg       0.78      0.78      0.78     77400



#### 4. Использование другой функции извлечения признаков из имени

Попробуем использовать разные функции и посмотреть на метрики

In [231]:
def do_everything(feature_extraction_func: Callable, clf_func: Callable):
    clf = NaiveBayesClassifier(feature_extraction_func, clf_func)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred, 
                                target_names=sex_encoder.inverse_transform([0, 1])))    

Признаки - первая и последняя буква

In [232]:
do_everything(lambda n: (n[0], n[-1]), clf_func_1)

              precision    recall  f1-score   support

         boy       0.76      0.81      0.79     38668
        girl       0.80      0.75      0.77     38732

    accuracy                           0.78     77400
   macro avg       0.78      0.78      0.78     77400
weighted avg       0.78      0.78      0.78     77400



Метрики по этой модели сталее более однородными (значения для мальчиков и девочек стали ближе друг к другу). Но прироста в качестве модель не получила

Признаки - 2 последние буквы

In [233]:
do_everything(lambda n: (n[-2], n[-1]), clf_func_1)

              precision    recall  f1-score   support

         boy       0.73      0.63      0.68     38668
        girl       0.67      0.77      0.72     38732

    accuracy                           0.70     77400
   macro avg       0.70      0.70      0.70     77400
weighted avg       0.70      0.70      0.70     77400



Метрики стали сильно хуже. Скорее всего потому, что пары последних букв почти все уникальные. Из-за этого прогноз ближе к константе - распределении мальчик/девочка в тестовой выборке.

Чтобы показать это можем вывести прогноз для имени целиком

In [234]:
do_everything(lambda n: list(n), clf_func_1)

              precision    recall  f1-score   support

         boy       0.74      0.56      0.64     38668
        girl       0.65      0.80      0.72     38732

    accuracy                           0.68     77400
   macro avg       0.69      0.68      0.68     77400
weighted avg       0.69      0.68      0.68     77400



#### 5. Используем другую функцию для классификации

In [235]:
def clf_func_2(classes, freqs, features):
    max_res, max_class = -1e9, None
    
    for label in classes:

        res = classes[label]

        for feat in features:
            if freqs[label, feat] == 0:  # handle lack of data
                return max(classes, key=lambda l: classes[l])
            
            res += freqs[label, feat]

        if res > max_res:
            max_class = label
            max_res = res
            
    return max_class

#### 6. Смотрим на результаты

In [236]:
do_everything(extr_func_last_letter, clf_func_2)

              precision    recall  f1-score   support

         boy       0.75      0.84      0.79     38668
        girl       0.82      0.72      0.77     38732

    accuracy                           0.78     77400
   macro avg       0.78      0.78      0.78     77400
weighted avg       0.78      0.78      0.78     77400



Видим, что метрики почти не поменялись (**если сранивать со значениями где брали последнюю букву**). Это связано с тем, что в первом примере считается энтропия, а во втором - критерий Джини

Но так вышло, что функции энтропии и $2*gini$ хорошо аппроксимируют друг друга на интервале вероятностей от 0 до 1. Поэтому отличия незначительны

#### 7. Классификаторы из sklearn

Прежде чем обучать классификаторы, текстовые данные надо превратить в числа, так как встроенные в sklearn классификаторы не умеют в буквы(

Любая буква может быть закодирована в ASCII. Поэтому возьмем как признак - код последней буквы + отмасштабируем, начав отсчет с нуля

In [237]:
X_train_vec = X_train.apply(lambda n: np.array(ord(n[-1]) - ord('a'))).to_numpy().reshape(-1, 1)
X_test_vec = X_test.apply(lambda n: np.array(ord(n[-1]) - ord('a'))).to_numpy().reshape(-1, 1)

Теперь обучим классификаторы из sklearn

In [238]:
from sklearn.naive_bayes import GaussianNB

gauss = GaussianNB()
gauss.fit(X_train_vec, y_train)

pred = gauss.predict(X_test_vec)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73     38668
           1       0.73      0.75      0.74     38732

    accuracy                           0.74     77400
   macro avg       0.74      0.74      0.74     77400
weighted avg       0.74      0.74      0.74     77400



In [239]:
from sklearn.naive_bayes import MultinomialNB

multi = MultinomialNB()
multi.fit(X_train_vec, y_train)

pred = multi.predict(X_test_vec)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67     38668
           1       0.00      0.00      0.00     38732

    accuracy                           0.50     77400
   macro avg       0.25      0.50      0.33     77400
weighted avg       0.25      0.50      0.33     77400



/home/vctrius/Code/iad_homeworks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vctrius/Code/iad_homeworks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vctrius/Code/iad_homeworks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

Тут выдаются ошибки деления на ноль. Ради интереса посмотрим на предикты

In [240]:
pred

array([0, 0, 0, ..., 0, 0, 0])

А там все нули))

Погуглив про мультиномиальное распределение пришел к выводу что можно попробовать брать больше чем один признак

Тогда можно взять 2 последних буквы

In [241]:
X_train_vec = pd.concat([X_train.str.lower().str[-1].apply(ord), 
                         X_train.str.lower().str[-2].apply(ord)], 
                         axis=1) - ord('a')
X_test_vec = pd.concat([X_test.str.lower().str[-1].apply(ord), 
                         X_test.str.lower().str[-2].apply(ord)], 
                         axis=1) - ord('a')

In [242]:
multi = MultinomialNB()
multi.fit(X_train_vec, y_train)

pred = multi.predict(X_test_vec)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73     38668
           1       0.73      0.73      0.73     38732

    accuracy                           0.73     77400
   macro avg       0.73      0.73      0.73     77400
weighted avg       0.73      0.73      0.73     77400



Метрики у этих классификаторов получились хуже, чем у наивного. Но зато появилась стабильность (метрика почти не зависит от класса)

## Задание 2. Ирисы

#### 0. Загрузим данные

In [243]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris['data'], iris['target']

#### 1. Разделим выборку

In [244]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 2. Обучим LDA (встроенный в sklearn)

In [245]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis('eigen')
lda.fit(X_train, y_train);

pred = lda.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



Видим, что встроенный классификатор дает ровно 0 ошибок, потому что он умный и делает все правильно. Теперь посмотрим на LDA из семинара

#### 3. LDA из семинара

In [246]:
def LDA_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    #St = np.dot((X - mju).T, X - mju)

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                     X_classify[i] - mju_classify[i])

    # Sb=St-Sw

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
            (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs

Судя по документации sklearn, `LinearDiscriminantAnalysis` это сам LDA (который позволяет уменьшить размерность) + наивный байесовский классификатор

Поэтому LDA возьемем из семинара, а классификатором восполбзуемся встроенным

In [247]:
w = LDA_dimensionality(X_train, y_train, 1)
X_train_lda = np.dot(X_train, w)
X_test_lda = np.dot(X_test, w)

lda = GaussianNB()
lda.fit(X_train_lda, y_train);

pred = lda.predict(X_test_lda)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



И опять единичка) Причем при любом количестве выходных признаков (даже 1)

#### 4. Изменение параметров классификатора

Так как встроенный классификатор и так работает идеально, эффективней его делать некуда. Поэтому попробуем понизить ему метрику

In [248]:
for solv in ['eigen', 'lsqr', 'svd']:
    lda = LinearDiscriminantAnalysis(solv)
    lda.fit(X_train, y_train);

    pred = lda.predict(X_test)
    print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy        

Видим, что датасет настолько базовый, что любой солвер выдает единицу

Попробуем поиграться со специфичными для каждого солвера параметрами

In [249]:
params = [{'solver': 'svd', 'store_covariance': True},
           {'solver': 'svd', 'tol': 0.1},
           {'solver': 'lsqr', 'shrinkage': 'auto'}]

for p in params:
    lda = LinearDiscriminantAnalysis(**p)
    lda.fit(X_train, y_train);

    pred = lda.predict(X_test)
    print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy        

К сожалению, ничего не помогает)

При любом солвере (сингулярное разложение, через МНК, разложение на собственные значения) ничего не меняется. Также изменения специфичных параметров, таких как `store_covariance` (сохранение матрицы ковариаций), изменение порога `tol` на несколько порядков в обе стороны, а также автоматическое сокращение `shrinkage` метрику не меняют

Слишком удачный датасет